
# 4 - IP Build RCAs from Pre-Processed NetMap Reaches and DEM

Use this tool to build reach contributing area (RCAs) from a TauDEM and NHDPlus stream reaches and a digital elevation model (DEM). This tool is designed to replace the STARS "Generate Cost RCAs" tool which was built using Python 2 and designed for use in ArcMap.

## Required Software:

- This code is formatted as a Jupyter notebook markdown document and can be modified to be run in ARCGIS Pro or any python gui
- If any of the geoprocessing steps require an advanced license or any specific extensions, the script will check for these conditions before running.

## Required Inputs:

- DEMs used to create the synthetic stream networks and NHDPlus data.   

- Stream reaches and NHDPlus flowlines

- A unique attribute field catchment id or nhdplus id

- A feature class name for the RCA polygon output.

- A system directory to use as a temporary file location. It is recommended that this be a short file path on an internal hard drive (eg, "C:/data").


## Geoprocessing Output:

- An RCA polygon feature class written to the default project geodatabase.

## Processing Steps:

1. Convert the pre-processed reaches to a raster, using the same cell size and extent as the input DEM (May already have this raster ).
2. Create a raster object from the rasterized streamlines.
3. Run a series of Raster Calculator expressions. (These calculations essentially create a cost surface where ridgeline landscape positions become very expensive compared to valley and hillslope positions. This will constrain the RCA polygons and prevent them from crossing ridgetops into adjacent drainages.)
4. Run the Cost Allocation using the rasterized stream reaches and the final cost surface.
5. Build a raster attribute table for the cost allocation output.
6. Convert the cost allocation output to polygon, and dissolve any multi-part polygons.
7. Add a field for the RCA ID, then calculate the RCA ID to equal the "gridcode" attribute from the original rasterized reaches.
8. Delete all intermediary rasters.

### Code starts here:

#### Setup

Import modules and reset environments to default. This should set the ArcPro project geodatabase as the workspace/scratch environment, just in case it was set otherwise. Additionally, prevent the addition of intermediary outputs to the ArcPro project map. Some tools may not run if their target is open in the map display.

In [1]:
import os, arcpy, sys,datetime, traceback

import arcpy.management

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
print(f'Date: {datetime.datetime.now()}')
print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())
outdir = r"W:\\GIS\\AKSSF_ValBot_2024V3"
outgdbname = 'AKSSF_ValBotV3.gdb'
outgdb = os.path.join(outdir,outgdbname)


# Check if the output directory exists, and create it if not
if not os.path.exists(outdir):
    os.makedirs(outdir)
    print(f'Output directory "{outdir}" created')

# Check if the output geodatabase exists, and create it if not
if not arcpy.Exists(outgdb):
    arcpy.management.CreateFileGDB(outdir, outgdbname)
    print(f'Output geodatabase "{outgdb}" created')

print(f'Output directory set to {outdir}')

print(f'Output gdb set to {outgdb}')


Date: 2024-02-21 09:25:05.770181
imports complete
----------------------------------------------------------------------------------------------------
sys paths ['S:\\Github\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology\\confinement_scripts', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcPy', 'S:\\Github\\AKSSF', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python39.zip', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Program Files\\ArcGIS\\Pro\\bin', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcToolbox\\Scripts', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages\\Babel-2.11.0-py3.9.egg', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-p

The Spatial Analyst extension is required for this script to work. Check for extension, and if its available check it out for use. If it is unavailable or cannot be checked out, throw a license error and provide an error message.

In [2]:
print("Spatial Analyst license is required.")
arcpy.AddMessage("Spatial Analyst license is required.")
print(arcpy.GetMessages())

class LicenseError(Exception):
    pass

try:
    if arcpy.CheckExtension("Spatial") == "Available":
        arcpy.CheckOutExtension("Spatial")
        print("Good news...Spatial Analyst license is available!")
        arcpy.AddMessage("Good news...Spatial Analyst license is available!")
        print(arcpy.GetMessages())
    else:
        # raise a custom exception
        raise LicenseError

except LicenseError:
    print("Bad news...Spatial Analyst license is unavailable.")
    arcpy.AddMessage("Bad news...Spatial Analyst license is unavailable.")
    print(arcpy.GetMessages())
except arcpy.ExecuteError:
    print(arcpy.GetMessages())

Spatial Analyst license is required.

Good news...Spatial Analyst license is available!


In [ ]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput

# Getnull rows from numpy array
def getnull(cat_ID_con):
    nullRows = []
    nullRows.append(cat_ID_con)
    return True

#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

## Set input Data
Set input datasets and data dictionaries


In [4]:
import time
import datetime
import os
import arcpy

prefDict = {'Bristol_Bay':'bb', 'Kodiak':'kod', 'Prince_William_Sound':'pws','Cook_Inlet':'ci','Copper_River':'cr'}

# Separate data by source type
nhdplus_dat = ['Cook_Inlet', 'Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Set input regional folders
#regions = [f'W:\\GIS\\AKSSF\\{source}' for source in nhdplus_dat + tauDem_dat]

regions = [f'W:\\GIS\\AKSSF\\Bristol_Bay']

strDict = {}
roiRasDict = {}
catsDict = {}

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()
print(f'Begin {datetime.datetime.now()}')

for region in regions:
    roi = os.path.basename(region)
    print(roi)

    arcpy.env.workspace = region
    gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
    print(f'GDB {gdb}')

    walk = arcpy.da.Walk(region, datatype=['FeatureClass', 'RasterDataset'])
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            if filename == 'NHDFlowline_merge' and roi in nhdplus_dat:
                streamname = f'{prefDict[roi]}_{filename}'
                streams = os.path.join(outgdb, streamname)
                if not arcpy.Exists(streams):
                    print(f'Copying {os.path.join(dirpath, filename)} to {outgdb}')
                    startTime = time.time()  # Start timer
                    # Select only streams/rivers and artificial paths
                    arcpy.FeatureClassToFeatureClass_conversion(
                        in_features=os.path.join(dirpath, filename),
                        out_path=outgdb,
                        out_name=streamname,
                        where_clause="FType = 460 Or FType = 558"
                    )
                    endTime = time.time()  # End timer
                    elapsedTime = endTime - startTime  # Calculate elapsed time
                    print(f'Copy completed in {elapsedTime} seconds')
                else:
                    print(f'{streams} already copied')
                append_value(strDict, roi, [streams, 'NHDPlusID'])
                
            elif filename == 'streams_merge' and roi in tauDem_dat:
                if roi == 'Bristol_Bay':
                    id = 'catID'
                else:
                    id = 'LINKNO'

                streamname = f'{prefDict[roi]}_{filename}'
                streams = os.path.join(outgdb, streamname)
                if not arcpy.Exists(streams):
                    print(f'Copying {os.path.join(dirpath, filename)} to {outgdb}')
                    startTime = time.time()  # Start timer
                    arcpy.FeatureClassToFeatureClass_conversion(
                        os.path.join(dirpath, filename),
                        outgdb,
                        streamname
                    )
                    endTime = time.time()  # End timer
                    elapsedTime = endTime - startTime  # Calculate elapsed time
                    print(f'Copy completed in {elapsedTime} seconds')
                else:
                    print(f'{streams} already created')
                append_value(strDict, roi, [streams, id])
            
            elif filename == 'cats_merge':
                if roi in nhdplus_dat:
                    catID = 'NHDPlusID'
                else:
                    catID = 'catID'
                    
                catsname = f'{prefDict[roi]}_{filename}'
                cats = os.path.join(outgdb, catsname)
                if not arcpy.Exists(cats):
                    print(f'Copying {os.path.join(dirpath, filename)} to {outgdb}')
                    startTime = time.time()  # Start timer
                    # Select only streams/rivers and artificial paths
                    arcpy.FeatureClassToFeatureClass_conversion(
                        in_features=os.path.join(dirpath, filename),
                        out_path=outgdb,
                        out_name=catsname,
                    )
                    endTime = time.time()  # End timer
                    elapsedTime = endTime - startTime  # Calculate elapsed time
                    print(f'Copy completed in {elapsedTime} seconds')
                else:
                    print(f'{cats} already copied')           
                append_value(catsDict, roi, [cats, catID])

            elif filename.endswith('.tif'):
                if filename == 'elev.tif':
                    raster_type = 'elev'
                elif filename == 'slope.tif':
                    raster_type = 'slope'
                elif filename == 'fac.tif':
                    raster_type = 'fac'
                elif filename == 'fdr.tif':
                    raster_type = 'fdr'
                else:
                    continue  # Skip if not the required raster

                raster_name = f'{prefDict[roi]}{raster_type}.tif'
                raster_path = os.path.join(outdir, raster_name)
                if not arcpy.Exists(raster_path):
                    print(f'Copying {os.path.join(dirpath, filename)} to {outdir}')
                    startTime = time.time()  # Start timer
                    arcpy.CopyRaster_management(os.path.join(dirpath, filename), raster_path)
                    endTime = time.time()  # End timer
                    elapsedTime = endTime - startTime  # Calculate elapsed time
                    print(f'Copy completed in {elapsedTime} seconds')
                else:
                    print(f'{raster_path} already created')
                append_value(roiRasDict, roi, [raster_path])

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success
print(f'{"*"*100}')
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')
print(strDict)
print(f'{"*"*100}')
print(roiRasDict)


Begin 2024-02-21 09:25:06.410676
Bristol_Bay
GDB ['W:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb']
W:\\GIS\\AKSSF_ValBot_2024V3\bbelev.tif already created
W:\\GIS\\AKSSF_ValBot_2024V3\bbfac.tif already created
W:\\GIS\\AKSSF_ValBot_2024V3\bbfdr.tif already created
W:\\GIS\\AKSSF_ValBot_2024V3\bbslope.tif already created
W:\\GIS\\AKSSF_ValBot_2024V3\AKSSF_ValBotV3.gdb\bb_cats_merge already copied
W:\\GIS\\AKSSF_ValBot_2024V3\AKSSF_ValBotV3.gdb\bb_streams_merge already created
****************************************************************************************************
Process completed at 2024-02-21 09:25 (Elapsed time: 0:00:02)
****************************************************************************************************
{'Bristol_Bay': ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\AKSSF_ValBotV3.gdb\\bb_streams_merge', 'catID']}
****************************************************************************************************
{'Bristol_Bay': ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\bbe

### Combine the dictionaries


In [5]:
combinedDict = {}

# Add key-value pairs from strDict to combinedDict
combinedDict = {
    x: roiRasDict.get(x, 0) + strDict.get(x, 0) + catsDict.get(x, 0)
    for x in set(roiRasDict.keys()).union(strDict.keys(), catsDict.keys())
}
combinedDict

{'Bristol_Bay': ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\bbelev.tif',
  ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\bbfac.tif'],
  ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\bbfdr.tif'],
  ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\bbslope.tif'],
  'W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\AKSSF_ValBotV3.gdb\\bb_streams_merge',
  'catID',
  'W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\AKSSF_ValBotV3.gdb\\bb_cats_merge',
  'catID']}

# Generate valley bottom polygons
Convert stream polylines to raster and generate valley bottom polygons
Some headwater streams have valleys that get broken up into "raster cell" chunks with disconnect sections...may need to create a buffer (10 meters?) on 1st order streams and merge with valley bottom polygon to fix.


In [6]:
import time, datetime

accThresh = 550 # Lower acc thresh to 550

def timer(processing_step, *args, **kwargs):
    try:
        # Start timing function
        processStart = time.time()
        processStartdt = datetime.datetime.now()
        print(f'Begin {processing_step.__name__}: {datetime.datetime.now()}')

        # Call the original geoprocessing function
        result = processing_step(*args, **kwargs)

        # End timing
        processEnd = time.time()
        processElapsed = int(processEnd - processStart)
        processSuccess_time = datetime.datetime.now()

        # Report success with the specified processing step name
        print(f'{processing_step.__name__} completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
              f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
        print(f'{"*"*100}')

        return result, processElapsed

    except Exception as e:
        # Handle exceptions
        processError_time = datetime.datetime.now()
        print(f'Error in {processing_step.__name__} at {processError_time.strftime("%Y-%m-%d %H:%M")}: {str(e)}')
        print(f'{"*"*100}')
        return None, None
    
    
# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()
print(f'Begin {datetime.datetime.now()}')

for k,v in combinedDict.items():
    if k == 'Bristol_Bay':
        reachRas = os.path.join(outdir,prefDict[k] + "rchRas.tif")
        vbPoly = os.path.join(outgdb,prefDict[k] + "_vbPoly")
        DEM_ras = arcpy.Raster(v[0])
        slope_ras = arcpy.Raster(v[3])
        size = DEM_ras.meanCellWidth
        size = int(size)
        reachid = v[5]
        streams = v[4]
        cats = v[6]
        catID = v[7]
        src_init_accum = 0 #Leave initial accumulation at 0 
        src_cost_mp = 1.90 #Increase the cost from 1.75 to 1.9
        arcpy.env.snapRaster = DEM_ras
        print(k,v)
        #reachRas should already exist...src raster no?
        if not arcpy.Exists(reachRas):
            print(f'Converting {streams} to raster')
            reachRas, elapsed_time = timer(arcpy.conversion.PolylineToRaster,streams,#streams
                                              reachid,#reachID
                                              reachRas,#reachRaster
                                              "",
                                              "",
                                              size,
                                              ""
                                           )
        else:
            print(f'{reachRas} already created')
        vbSmoothname = f'{prefDict[k]}vb{accThresh}Smooth'    
        vbSmooth = os.path.join(outgdb, vbSmoothname)
        if not arcpy.Exists(vbSmooth):
            outDistAcc, elapsed_time = timer(arcpy.sa.DistanceAccumulation,reachRas,'',DEM_ras,slope_ras,'','','','','','','',src_init_accum,'',src_cost_mp,'','')
            #outDistAcc, elapsed_time = timer(arcpy.sa.DistanceAccumulation,reachRas, "", "", slope_ras, "", "", "", "", "", "", "", "", "", 1.75, "", "")          
            DistAcc, elapsed_time = timer(arcpy.sa.Con,outDistAcc, 1, -123, f'VALUE <= {accThresh}')
            DistAcc_Nulled = arcpy.sa.SetNull(DistAcc, DistAcc, 'VALUE = -123')
            vbPoly, elapsed_time= timer(arcpy.conversion.RasterToPolygon, DistAcc_Nulled, vbPoly, "SIMPLIFY", "", "", "")
            vbSmooth, elapsed_time = timer(arcpy.cartography.SmoothPolygon,vbPoly, vbSmooth, "PAEK", 250) 
            arcpy.management.Delete(outDistAcc)
            arcpy.management.Delete(DistAcc)
            arcpy.management.Delete(DistAcc_Nulled)
        else:
            print(f'{vbSmooth} already generated')
        
        #Clean smoothed polygon
        vbCleanname =  f'{prefDict[k]}vb{accThresh}Clean'
        vbClean = os.path.join(outgdb,vbCleanname)
        if not arcpy.Exists(vbClean):
            print(f'Cleaning {vbSmooth} to remove polygon smoothing artifacts')
            pcnt1 = arcpy.GetCount_management(vbSmooth)
            vbLayer = arcpy.MakeFeatureLayer_management(vbSmooth)
            vbSelect = arcpy.management.SelectLayerByLocation(
                in_layer = vbSmooth,
                overlap_type="INTERSECT",
                select_features=streams,
                search_distance=None,
                selection_type="NEW_SELECTION",
                invert_spatial_relationship="NOT_INVERT"
            )
            pcnt2 = arcpy.GetCount_management(vbSelect)
            print(f'Selected {pcnt2} of {pcnt1} vb polygons')
            arcpy.FeatureClassToFeatureClass_conversion(vbSelect,outgdb,vbCleanname)
            append_value(combinedDict,k,vbClean)
        else:
            print(f'{vbClean} already created')
        
        #Intersect catchments with cleaned valley bottom polygons
        vbCleanIntname = prefDict[k] + f"vb{accThresh}CleanInt"
        vbCleanInt = os.path.join(outgdb, vbCleanIntname)
        vbCleanIntSpname = prefDict[k] + f"vb{accThresh}CleanIntSp"
        vbCleanIntSp = os.path.join(outgdb, vbCleanIntSpname)
        vbCleanIntSelectname = prefDict[k] + f"vb{accThresh}CleanIntSpSelect"
        vbCleanIntSelect = os.path.join(outgdb, vbCleanIntSelectname)
        if not arcpy.Exists(vbCleanIntSelect):
            print(f'Intersecting {cats} with {vbClean}')
            arcpy.analysis.PairwiseIntersect(
                in_features=[cats,vbClean],
                out_feature_class=vbCleanInt,
                join_attributes="ALL",
                cluster_tolerance=None,
                output_type="INPUT"             
            )
            #convert multipart to single part to identify any catchments split by the intersect operation
            print(f'Converting {vbCleanIntname} to single part feature')
            vbCleanIntSp, elapsed_time = timer(arcpy.MultipartToSinglepart_management,vbCleanInt,vbCleanIntSp)  
            print(f'Selecting intersected catchments using streams')
            vbIntSelect = arcpy.management.SelectLayerByLocation(
                in_layer = vbCleanIntSp,
                overlap_type="INTERSECT",
                select_features=streams,
                search_distance=None,
                selection_type="NEW_SELECTION",
                invert_spatial_relationship="NOT_INVERT"
            )
            vbCleanIntSelect = arcpy.FeatureClassToFeatureClass_conversion(vbIntSelect,outgdb,vbCleanIntSelectname)
            append_value(combinedDict,k,vbCleanIntSelect)
        else:
            print(f'{vbCleanIntSelect} already created')
        
        

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success
print(f'All Processes completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')




Begin 2024-02-14 08:59:51.324633
Bristol_Bay ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\bbelev.tif', ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\bbfac.tif'], ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\bbfdr.tif'], ['W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\bbslope.tif'], 'W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\AKSSF_ValBotV3.gdb\\bb_streams_merge', 'catID', 'W:\\\\GIS\\\\AKSSF_ValBot_2024V3\\AKSSF_ValBotV3.gdb\\bb_cats_merge', 'catID']
W:\\GIS\\AKSSF_ValBot_2024V3\bbrchRas.tif already created
Begin DistanceAccumulation: 2024-02-14 08:59:51.374727
DistanceAccumulation completed at 2024-02-14 09:16 (Elapsed time: 0:16:40)
****************************************************************************************************
Begin Con: 2024-02-14 09:16:32.212715
Con completed at 2024-02-14 09:17 (Elapsed time: 0:01:12)
****************************************************************************************************
Begin RasterToPolygon: 2024-02-14 09:18:25.645497
RasterToPolygon completed at 2024-02-14 09:19 (Elapsed tim

## Identify catchments split during the intersect process
Catchments that were split into multiple polygons when intersected with the valley bottom polygon must be reselected and cleaned
1. Intersect Streams with new catchment/vb intersect polygon
2. Convert Stream Intersect to Center points and Endpoints (shifted slightly downstream)
3. Merge two point layers together
4. Select from polygon layer all catchments that contain clementi the stream mid and endpoints.
5. Export and dissolve back on catchment ID

In [7]:
import time
import datetime
import math
import os
import arcpy

for k,v in combinedDict.items():
    if k == 'Bristol_Bay':
        streams = combinedDict[k][4]
        # Start timing function
        processStart = time.time()
        processStartdt = datetime.datetime.now()
        print(f'Begin {datetime.datetime.now()}')
        print(f'{"*"*100}')
        
        vbCleanname = f'{prefDict[k]}vb{accThresh}Clean'
        vbClean = os.path.join(outgdb, vbCleanname)
        vbFinalname = f'{prefDict[k]}vb{accThresh}Final'
        vbFinal = os.path.join(outgdb, vbFinalname)
        vbCleanPtSelectname = f'{prefDict[k]}vb{accThresh}CleanPtSelect'
        vbCleanPtSelect = os.path.join(outgdb, vbCleanPtSelectname)
        streamIntname = f'{prefDict[k]}vb{accThresh}strIntvbInt'
        streamInt = os.path.join(outgdb, streamIntname)
        vbCleanIntSelectname = f'{prefDict[k]}vb{accThresh}CleanIntSpSelect'
        vbCleanIntSelect = os.path.join(outgdb, vbCleanIntSelectname)
        strIntCtrPtname = f'{prefDict[k]}vb{accThresh}strIntCtrPts'
        strIntCtrPts = os.path.join(outgdb, strIntCtrPtname)
        strIntEndPtname = f'{prefDict[k]}vb{accThresh}strIntEndPts'
        strIntEndPts = os.path.join(outgdb, strIntEndPtname)
        strIntAllPtname = f'{prefDict[k]}vb{accThresh}strIntEndPts'
        strIntAllPts = os.path.join(outgdb, strIntAllPtname)
        
        # Intersect Streams with catchments/vb
        if not arcpy.Exists(streamInt):
            print(f'Intersecting streams with catchment/vb polygons')
            print(f'{"*"*100}')
            
            streamInt,elapsed_time = timer(arcpy.PairwiseIntersect_analysis,
                                           in_features=[streams,vbCleanIntSelect],
                                           out_feature_class=streamInt,
                                           join_attributes="ALL",
                                           cluster_tolerance=None,
                                           output_type="LINE"
                                           )
        else:
            print(f'{streamInt} already created')
            print(f'{"*"*100}')
            
        # Create Stream Intersect Polyine center points
        if not arcpy.Exists(strIntCtrPts):
            print(f'Creating stream center point feature class: {strIntCtrPts}')
            print(f'{"*"*100}')
            
            strIntCtrPts, elapsed_time = timer(arcpy.management.FeatureToPoint,
                                               in_features=streamInt,
                                               out_feature_class=strIntCtrPts,
                                               point_location="INSIDE"
                                               )
        else:
            print(f'{strIntCtrPts} already created')
            print(f'{"*"*100}')
            
        # Create point layer of stream endpoints shifted down the line by one meter
        if not arcpy.Exists(strIntEndPts):
            print(f'Creating shifted endpoints feature class')
            print(f'{"*"*100}')
            
            strIntEndPts = arcpy.CreateFeatureclass_management(outgdb,strIntEndPtname,'POINT')
            
            # Record the start time
            start_time = time.time()
           
            # Search cursor to iterate through polyline features
            with arcpy.da.SearchCursor(streamInt, ["SHAPE@"]) as cursor:
                for row in cursor:
                    polyline = row[0]
            
                    # Get the last point of the polyline (endpoint)
                    endpoint = polyline.lastPoint
            
                    # Specify the distance to slide the point (0.1 meters to the left)
                    offset_distance = -1.0  # negative for left
            
                    # Use the positionAlongLine method to slide the point
                    point = polyline.positionAlongLine(polyline.length + offset_distance)
            
                    # Create a point geometry
                    new_point = arcpy.Point(point.firstPoint.X, point.firstPoint.Y)
            
                    # Insert the new point into the output feature class
                    with arcpy.da.InsertCursor(strIntEndPts, ["SHAPE@"]) as insert_cursor:
                        insert_cursor.insertRow([new_point])
            
            # Record the end time
            end_time = time.time()           
            # Calculate the elapsed time
            elapsed_time = end_time - start_time
            print("Point generation complete.")
            print(f"Elapsed time: {elapsed_time} seconds")
            print(f'{"*"*100}')
            
        else: 
            print(f'{strIntEndPts} already created')
            print(f'{"*"*100}')
        
        #Merge stream center and offset endpoints together
        if not arcpy.Exists(strIntAllPts):
            print(f'Merging center and endpoints together')
            print(f'{"*"*100}')
            arcpy.Merge_management([strIntCtrPts,strIntEndPts],strIntAllPts,add_source='NO_SOURCE_INFO')
        else:
            print (f'{strIntAllPts} created')
            print(f'{"*"*100}')
            
        if not arcpy.Exists(vbCleanPtSelect):
            print(f'Selecting catchments from {vbCleanIntSelect}')
            print(f'{"*"*100}')
            
            vbSelectbyPoints = arcpy.management.SelectLayerByLocation(
            in_layer=vbCleanIntSelect,
            overlap_type="CONTAINS_CLEMENTINI",
            select_features=strIntAllPts,
            search_distance=None,
            selection_type="NEW_SELECTION",
            invert_spatial_relationship="NOT_INVERT"
                )
                             
            # Export Selected features
            arcpy.FeatureClassToFeatureClass_conversion(vbSelectbyPoints,
                                                        outgdb,
                                                        vbCleanPtSelectname                                                        
                                                        )
        else:
            print(f'{vbCleanPtSelect} already exists')
            print(f'{"*"*100}')
        
        #Dissolve selection back on catchment ID
        if not arcpy.Exists(vbFinal):
            arcpy.PairwiseDissolve_analysis(vbCleanPtSelect,vbFinal,combinedDict[k][7])
        else:
            print(f'{vbFinal} already created')
            print(f'{"*"*100}')
        
        # End timing
        processEnd = time.time()
        processElapsed = int(processEnd - processStart)
        processSuccess_time = datetime.datetime.now()
    
        print(f'\n{vbFinal} has {arcpy.GetCount_management(vbFinal)} features \n')
        print(f'All Processes completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
              f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
        print(f'{"*"*100}')


Begin 2024-02-14 09:36:03.251471
****************************************************************************************************
Intersecting streams with catchment/vb polygons
****************************************************************************************************
Begin PairwiseIntersect: 2024-02-14 09:36:03.267114
PairwiseIntersect completed at 2024-02-14 09:36 (Elapsed time: 0:00:53)
****************************************************************************************************
Creating stream center point feature class: W:\\GIS\\AKSSF_ValBot_2024V3\AKSSF_ValBotV3.gdb\bbvb550strIntCtrPts
****************************************************************************************************
Begin FeatureToPoint: 2024-02-14 09:36:56.760126
FeatureToPoint completed at 2024-02-14 09:37 (Elapsed time: 0:00:21)
****************************************************************************************************
Creating shifted endpoints feature class
*******************

# Calculate vb mean widths


In [3]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput

# Getnull rows from numpy array
def getnull(cat_ID_con):
    nullRows = []
    nullRows.append(cat_ID_con)
    return True

#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

#  New Method using transects to calculate vb widths
 
Extend transect lines well past valley basin polygon boundaries. 
Clip to vb basin and intersect
Dissolve and select again using original small transects to remove lines that may extend outside of basin and enter again


## Smooth streamlines and generate transects
Smooth streams using PAEK (50 meter) method to try and reduce jaggedness from raster derived streams


In [62]:
import arcpy
import os
import time
import datetime
arcpy.env.overwriteOutput = True

# Timer decorator
def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        print(f"{func.__name__} started {datetime.datetime.now()}")
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"{func.__name__} completed in {elapsed_time:.2f} seconds")
        print(f"{'*'*100}\n")
        return result
    return wrapper

# Decorate functions with the timer decorator
@timer
def smooth_lines(instreams, barriers, smoothstreamspath):
    if not arcpy.Exists(smoothstreamspath):
        print (f'Smoothing streams')
        arcpy.cartography.SmoothLine(
        in_features=instreams,
        out_feature_class=smoothstreamspath,
        algorithm="PAEK",
        tolerance="50 Meters",
        endpoint_option="FIXED_CLOSED_ENDPOINT",
        error_option="RESOLVE_ERRORS",
        in_barriers=barriers
            )
    else:
        print(f'{smoothstreamspath} already created')
    return smoothstreamspath

@timer
def generate_small_transects(smooth_streams, tempsmalltrans, smalltransname):
    if not arcpy.Exists(smalltrans):
        print (f'Generating selection transects')
        arcpy.management.GenerateTransectsAlongLines(
            in_features=smooth_streams,
            out_feature_class=tempsmalltrans,
            interval="50 Meters",
            transect_length="1 Meters",
            include_ends="NO_END_POINTS"
            )
        # Some transects still cross the catchment boundary even at 0.5 meters long. 
        # Add select by location and switch selection on transects that intersect catchment boundaries
        # and drop these from the set.
        smalltransselect =  arcpy.management.SelectLayerByLocation(
            in_layer=tempsmalltrans,
            overlap_type="COMPLETELY_WITHIN",
            select_features=cats,
            selection_type="NEW_SELECTION",
            invert_spatial_relationship="NOT_INVERT"
            )
        arcpy.FeatureClassToFeatureClass_conversion(smalltransselect,outgdb,smalltransname)
        #arcpy.Delete_management(tempsmalltrans)
    else:
        print(f'{smalltrans} already created')

@timer
def generate_transects(smooth_streams, transname, extension_length):
    if not arcpy.Exists(transects):
        print (f'Generating transects')
        arcpy.management.GenerateTransectsAlongLines(
            in_features=smooth_streams,
            out_feature_class=os.path.join(outgdb, transname),
            interval="50 Meters",
            transect_length=f"{str(extension_length)} Meters",
            include_ends="NO_END_POINTS"
            )
    else:
        print(f'{transects} already created')
       
for k,v in combinedDict.items():
    if k == 'Bristol_Bay':
        #Input parameters
        #outgdb = r'W:\GIS\AKSSF_ValBot_2023\AKSSF_ValBot.gdb'
        instreams = combinedDict[k][4]
        cats = combinedDict[k][6]
        smoothstreamsname =f'{prefDict[k]}vb{accThresh}smoothPAEK50Topo'
        smoothstreamspath = os.path.join(outgdb, smoothstreamsname)
        barriers = os.path.join(outgdb, f'{prefDict[k]}vb{accThresh}Final')
        #extension_length = float(input("Enter the extension length (default is 5000): ") or 5000)
        extension_length = 20000
        tempsmalltransname = f'{prefDict[k]}vb{accThresh}smallTransTemp'
        tempsmalltrans = os.path.join(outgdb,tempsmalltransname)
        smalltransname = f'{prefDict[k]}vb{accThresh}smallTrans'
        smalltrans = os.path.join(outgdb,smalltransname)
        transname = (f'{prefDict[k]}vb{accThresh}trans{str(extension_length).replace(".0","")}'
                     f'')
        transects = os.path.join(outgdb, transname)
        
        # Execute decorated functions with user input
        smooth_lines(instreams, barriers, smoothstreamspath)
        generate_small_transects(smoothstreamspath, tempsmalltransname,smalltransname)
        generate_transects(smoothstreamspath,transname, extension_length)
        
       


generate_transects started 2024-02-21 12:43:28.830464
Generating transects
generate_transects completed in 1629.26 seconds
****************************************************************************************************


## Generate Transects

wow! this used memory workspace and seemed to utilize all cores with some throttling
Script started at 2023-12-13 10:54:07
pairwise_intersect_features started 2023-12-13 10:54:07.660705
pairwise_intersect_features completed in 534.86 seconds
****************************************************************************************************

Randomly getting ExecuteError: ERROR 160385: Workspace or data source is read only.
Failed to execute (PairwiseIntersect).
Tried different drives and memory workspace? Will try on another computer.  Maybe need to free up more RAM?
2024 - Got process to work on dual processor computer  - sometimes deleting gdb and running fresh from start would work too
****************************************************************************************************
2024 - 02/05
Now select and copy running without completion for days, trying multiprocessing and failing that will try fishnet and merge


In [1]:
import multiprocessing
num_cores = multiprocessing.cpu_count()

print("Number of CPU cores:", num_cores)

Number of CPU cores: 32


In [ ]:
import arcpy
import os


arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = 100

accThresh = 550 # Lower acc thresh to 550
extension_length = 20000

# Timer decorator
def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        print(f"{func.__name__} started {datetime.datetime.now()}")
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"{func.__name__} completed in {elapsed_time:.2f} seconds")
        print(f"{'*'*100}\n")
        return result
    return wrapper

# Check existing funcs
# @timer
# def pairwise_intersect_features(features, output_fc):
#     if not arcpy.Exists(output_fc):
#         print(f"Creating {output_fc}. {__name__}.{pairwise_intersect_features.__name__} starting at {time.strftime('%Y-%m-%d %H:%M:%S')}")
#         arcpy.analysis.PairwiseIntersect(in_features=features, out_feature_class=output_fc, join_attributes="ALL", cluster_tolerance=None, output_type="LINE")
#     else:
#         print(f"{output_fc} already exists.")
# 
# @timer
# def mp2sp(input_intersection, out_mp):
#     if not arcpy.Exists(out_mp):
#         print(f"Creating {out_mp}. {__name__}.{mp2sp.__name__} starting at {time.strftime('%Y-%m-%d %H:%M:%S')}")
#         arcpy.management.MultipartToSinglepart(in_features=input_intersection, out_feature_class=out_mp)
#     else:
#         print(f"{out_mp} already exists.")
#     
# @timer
# def select_by_location(in_layer, overlap_type, select_features, selection_type, invert_spatial_relationship):
#     result = arcpy.management.SelectLayerByLocation(
#         in_layer=in_layer, overlap_type=overlap_type, select_features=select_features,
#         selection_type=selection_type, invert_spatial_relationship=invert_spatial_relationship
#     )
#     return result
# 
# @timer
# def copy_selected_features(input_fc, output_fc):
#     if not arcpy.Exists(output_fc):
#         print(f" Creating {output_fc}. {__name__}.{copy_selected_features.__name__} starting at {time.strftime('%Y-%m-%d %H:%M:%S')}")
#         arcpy.management.CopyFeatures(input_fc, output_fc)
#     else:
#         print(f"{output_fc} already exists.")
# 
# @timer
# def select_and_copy_features(input_fc, output_fc, select_features):
#     if not arcpy.Exists(output_fc):
#         selected_lines = select_by_location(input_fc, "SHARE_A_LINE_SEGMENT_WITH", select_features, "NEW_SELECTION", "NOT_INVERT")
#         copy_selected_features(selected_lines, output_fc)
#     else:
#         print(f'{output_fc} already exists')

# Check if a feature class has a spatial index
# def has_spatial_index(feature_class):
#     desc = arcpy.Describe(feature_class)
#     return desc.hasSpatialIndex
# 
# # Create a spatial index for a feature class if it doesn't already have one
# def create_spatial_index(feature_class):
#     if not has_spatial_index(feature_class):
#         arcpy.AddSpatialIndex_management(feature_class)
# 
# # Copy features to an in-memory feature class
# def copy_to_memory(input_fc, output_fc):
#     memory_workspace = "in_memory"
#     temp_output_fc = os.path.join(memory_workspace, output_fc)
#     arcpy.CopyFeatures_management(input_fc, temp_output_fc)
#     return temp_output_fc

# @timer
# # Combine selection and copy features steps
# def select_and_copy_features_parallel(input_fc, output_fc, select_features, num_processes=32):
#     # Check and create spatial index if necessary
#     create_spatial_index(input_fc)
#     create_spatial_index(select_features)
#     
#     # Copy input feature class to memory
#     input_memory_fc = copy_to_memory(input_fc, "input_memory_fc")
#     
#     # Function to select features by location
#     def select_by_location_wrapper(input_fc, select_features):
#         return arcpy.SelectLayerByLocation_management(input_fc, "SHARE_A_LINE_SEGMENT_WITH", select_features, "NEW_SELECTION", "NOT_INVERT")
#     
#     # Function to copy selected features
#     def copy_selected_features_wrapper(selected_lines, output_fc):
#         return arcpy.CopyFeatures_management(selected_lines, output_fc)
#     
#     # Perform selection and copying in parallel
#     with Pool(processes=num_processes) as pool:
#         partial_select_by_location = partial(select_by_location_wrapper, input_fc=input_memory_fc, select_features=select_features)
#         partial_copy_selected_features = partial(copy_selected_features_wrapper, output_fc=output_fc)
#         
#         selected_lines = pool.map(partial_select_by_location, [select_features] * num_processes)
#         output_fcs = pool.map(partial_copy_selected_features, selected_lines)
#     
#     # Merge output feature classes
#     arcpy.Merge_management(output_fcs, output_fc)
#     
#     # Delete intermediate data
#     arcpy.Delete_management(input_memory_fc)

@timer
def pairwise_intersect_features(features, output_fc):
    return arcpy.analysis.PairwiseIntersect(in_features=features, out_feature_class=output_fc, join_attributes="ALL", cluster_tolerance=None, output_type="LINE")

@timer
def mp2sp(input_intersection, out_mp):
    return arcpy.management.MultipartToSinglepart( in_features = input_intersection, out_feature_class=out_mp)

@timer
def select_by_location(in_layer, overlap_type, select_features, selection_type, invert_spatial_relationship):
    return arcpy.management.SelectLayerByLocation(in_layer=in_layer, overlap_type=overlap_type, select_features=select_features, selection_type=selection_type, invert_spatial_relationship=invert_spatial_relationship)

@timer
def copy_selected_features(input_fc, output_fc):
    return arcpy.management.CopyFeatures(input_fc, output_fc)

# Combine selection and copy features steps
@timer
def select_and_copy_features(input_fc, output_fc, select_features):
    selected_lines = select_by_location(input_fc, "SHARE_A_LINE_SEGMENT_WITH", select_features, "NEW_SELECTION", "NOT_INVERT")
    copy_selected_features(selected_lines, output_fc)


for k,v in combinedDict.items():
    if k == 'Bristol_Bay':
        # Input parameters
        vbFinalname = f'{prefDict[k]}vb{accThresh}Final'
        vbFinal = os.path.join(outgdb, vbFinalname)
        smalltransname = f'{prefDict[k]}vb{accThresh}smallTrans'
        transname = f'{prefDict[k]}vb{accThresh}trans{str(extension_length).replace(".0","")}'
        smalltrans = os.path.join(outgdb,smalltransname)
        transects = os.path.join(outgdb, transname)
        transIntname = f'{prefDict[k]}vb{accThresh}transInt'
        transIntpath = os.path.join(outgdb, transIntname)
        transmp2spname = f'{prefDict[k]}vb{accThresh}transIntSp'
        transmp2sppath = os.path.join(outgdb, transmp2spname)
        finaltransname = f'{prefDict[k]}vb{accThresh}TransectsFinal'
        finaltranspath = os.path.join(outgdb,finaltransname)
        # Print the start time
        print(f"Script started at {time.strftime('%Y-%m-%d %H:%M:%S')}")     
        
        arcpy.env.workspace = outgdb
        arcpy.env.overwriteOutput = True
        arcpy.env.parallelProcessingFactor = "100%"
        
        # Intersect clipped transects with valley bottom polygons using the pairwise tool
        transInt = pairwise_intersect_features([transects, vbFinal], transIntpath)
        
        #Convert multipart intersect to singlepart
        mp2spout = mp2sp(transIntpath, transmp2sppath)
        
        # Select lines that share a line segment with the original input transect feature class
        #select_and_copy_features_parallel(transmp2sppath, finaltranspath, smalltrans, num_processes=32) #not working
        
        #select_and_copy_features(transmp2sppath, finaltranspath, smalltrans)
        
        #Select and copy still not running-could try spatial join on pairwise intersect to get OIDs - sj smalltrans to transintpath using share a line segment 
        
        # Print the end time
        print(f"Script completed at {time.strftime('%Y-%m-%d %H:%M:%S')}")

Script started at 2024-02-21 13:10:38
pairwise_intersect_features started 2024-02-21 13:10:38.155362
pairwise_intersect_features completed in 5694.11 seconds
****************************************************************************************************

mp2sp started 2024-02-21 14:45:32.297478
mp2sp completed in 4449.46 seconds
****************************************************************************************************

select_and_copy_features started 2024-02-21 15:59:41.813829
select_by_location started 2024-02-21 15:59:41.813829


## Select and copy running without completing for an unknown reason.
Try running pairwise intersect on single part intersection with small transects.  Create list of feature ID's from intersection and use to write geometries to empty transect feature class with an update cursor.
  


In [36]:
#import arcpy

# Define the path to the feature class
pwint = r'W:\GIS\AKSSF_ValBot_2023\VB_QC\VB_QC.gdb\bbvb550transIn_PairwiseInter'

# Get field names and data types
field_info = [(field.name, field.type) for field in arcpy.ListFields(pwint)]

# Print field names and data types
print("Field Names and Data Types:")
print("\n".join([f"{name}: {data_type}" for name, data_type in field_info]))

# Iterate over the first 10 records
with arcpy.da.SearchCursor(pwint, [name for name, _ in field_info]) as cursor:
    for count, row in enumerate(cursor, start=1):
        print(f"\nRecord {count}:")
        for name, value in zip([name for name, _ in field_info], row):
            print(f"{name}: {value}")
        print("-" * 20)
        if count >= 10:
            break



Field Names and Data Types:
OBJECTID: OID
Shape: Geometry
FID_bbvb550transIntSp: Integer
FID_bbvb550smallTrans: Integer
Shape_Length: Double

Record 1:
OBJECTID: 1
Shape: (63803.38635000028, 1199611.6327499999)
FID_bbvb550transIntSp: 169378
FID_bbvb550smallTrans: 242725
Shape_Length: 0.9999810509638793
--------------------

Record 2:
OBJECTID: 2
Shape: (63765.725150000304, 1199642.3553500008)
FID_bbvb550transIntSp: 169383
FID_bbvb550smallTrans: 242726
Shape_Length: 0.9999904102827609
--------------------

Record 3:
OBJECTID: 3
Shape: (63721.237600000575, 1199661.99565)
FID_bbvb550transIntSp: 169449
FID_bbvb550smallTrans: 242727
Shape_Length: 0.9999933248010959
--------------------

Record 4:
OBJECTID: 4
Shape: (63835.70880000014, 1199581.219850001)
FID_bbvb550transIntSp: 171820
FID_bbvb550smallTrans: 233297
Shape_Length: 1.000020703930038
--------------------

Record 5:
OBJECTID: 5
Shape: (64227.20609999914, 1200118.3524000002)
FID_bbvb550transIntSp: 173136
FID_bbvb550smallTrans: 24287

In [57]:
pwintDict = {}
with arcpy.da.SearchCursor(pwint, ['FID_bbvb550smalltrans','FID_bbvb550transIntSp','Shape_Length']) as cursor:
    for row in cursor:
        k = row[0]
        v = row[1:]
        append_value(pwintDict,k,v)
pwintDict

{242725: (169378, 0.9999810509638793),
 242726: (169383, 0.9999904102827609),
 242727: (169449, 0.9999933248010959),
 233297: (171820, 1.000020703930038),
 242875: (173136, 1.000076977071889),
 242877: (173140, 0.9999483266864111),
 242876: (173143, 1.000017050025827),
 242757: (173288, 1.0000013854795538),
 229647: (173821, 0.9999541002209952),
 229648: (173823, 1.0000613683104216),
 229649: (173825, 1.0000726813392509),
 229650: (173827, 1.0000164651782528),
 242673: (181127, 0.9999515235937128),
 242674: (181129, 1.0000115199392101),
 242675: (181131, 1.0000213478470996),
 242676: (181133, 1.000021800477694),
 242677: (181137, 1.0000181252388889),
 235564: (185855, 0.9999907689689544),
 235565: (185860, 0.9999907689689544),
 235566: (185863, 0.9999570007173259),
 235567: (185866, 1.0000428440502507),
 235568: (185867, 0.9999386274780726),
 235569: (185868, 1.0000134489234178),
 235570: (185869, 1.0000713974779596),
 235571: (185870, 1.0000094528044992),
 235572: (185871, 0.999964945

In [40]:
pwintDict[242726]


169383

In [58]:

# Iterate through the dictionary and print key-value pairs for keys with more than one value
for key, values in pwintDict.items():
    if isinstance(values, list) and len(values) > 15:
        print(f"Key: {key}, Values: {values}")

Key: 608435, Values: [(95420434, 0.9999904102827609), (95420630, 0.9999904102827609), (95420846, 0.9999904102827609), (95421332, 0.9999904102827609), (95421339, 0.9999904102827609), (95422005, 0.9999904102827609), (95422025, 0.9999904102827609), (95422519, 0.9999904102827609), (95422550, 0.9999904102827609), (95423191, 0.9999904102827609), (95423206, 0.9999904102827609), (95423333, 0.9999904102827609), (95423389, 0.9999904102827609), (95423497, 0.9999904102827609), (95423577, 0.9999904102827609), (95423976, 0.9999904102827609), (95423992, 0.9999904102827609)]
Key: 608447, Values: [(95420690, 0.9999904102827609), (95420939, 0.9999904102827609), (95421047, 0.9999904102827609), (95421634, 0.9999904102827609), (95422146, 0.9999904102827609), (95422223, 0.9999904102827609), (95422571, 0.9999904102827609), (95422604, 0.9999904102827609), (95422619, 0.9999904102827609), (95422780, 0.9999904102827609), (95422840, 0.9999904102827609), (95422863, 0.9999904102827609), (95422956, 0.999990410282760

In [ ]:
import arcpy
import time
finaltrans = os.path.join(outgdb,f'{finaltransname}V3')
@timer
def intersect_and_update_fc(inmultipart, insmalltrans, outputfinaltrans):
    try:
        # Perform pairwise intersect between input feature classes
        intersect_output = arcpy.Intersect_analysis([inmultipart, insmalltrans], "MEMORY/intersect_output")

        # Create a dictionary of feature IDs and catchment values from the intersect output
        fid_catchment_dict = {}
        with arcpy.da.SearchCursor(intersect_output, [f'FID_{transmp2spname}', "catID", "SHAPE@"]) as cursor:
            for oid, catchment, geom in cursor:
                fid_catchment_dict[oid] = {'catchment': catchment, 'geometry': geom}

        # Create an empty polyline feature class for the final output
        arcpy.CreateFeatureclass_management(arcpy.env.workspace, outputfinaltrans, "POLYLINE", spatial_reference=inmultipart)

        # Use batch insertion with InsertCursor
        batch_size = 10000  # Adjust the batch size as needed
        with arcpy.da.InsertCursor(outputfinaltrans, ["SHAPE@", "catID"]) as cursor_out:
            features_to_insert = []
            for oid, data in fid_catchment_dict.items():
                geom = data['geometry']
                catchment = data['catID']
                features_to_insert.append([geom, catchment])
                if len(features_to_insert) == batch_size:
                    cursor_out.insertRow(features_to_insert)
                    features_to_insert = []
            # Insert any remaining features
            if features_to_insert:
                cursor_out.insertRow(features_to_insert)

        print("Intersect and update operation completed successfully.")
    except Exception as e:
        print(f"Error: {str(e)}")


# Example usage
intersect_and_update_fc("smalltrans", "transmp2sppath", "finaltrans")



Summarize valley widths by catchment and watershed?